In [1]:
from pytential.sympy_pytential import sympy_pytential
from pytential.reduce import min_pytential, pyt_minimizer

In [2]:
fa = sympy_pytential.from_properties({'mu0':[2,0]}, {'T':.300}, suffix='a')
fb = sympy_pytential.from_properties({'mu0':[0,2]}, {'T':.300}, suffix='b')

Density, rho, not found. Assuming rho = 1
Specific volumes, vi, not found. Assuming vi = 1/rho
Density, rho, not found. Assuming rho = 1
Specific volumes, vi, not found. Assuming vi = 1/rho


Make a collected function fa+fb with all the variables, and add constraints that the concentrations must sum to ca and cb

In [3]:
f = fa+fb
f = sympy_pytential.sum_extensive_variables(f, ['c0', 'c1'])
print(f)

vars ['Va', 'Vb', 'c0a', 'c0b', 'c1a', 'c1b']
prefix ['c0', 'c1']

Variables
['Va', 'Vb', 'c0', 'c0a', 'c0b', 'c1', 'c1a', 'c1b']

Potential
    ⎛          ⎛   c0a   ⎞    ⎞                 ⎛   c0b   ⎞                 ⎛ 
c0a⋅⎜2.4942⋅log⎜─────────⎟ + 2⎟ + 2.4942⋅c0b⋅log⎜─────────⎟ + 2.4942⋅c1a⋅log⎜─
    ⎝          ⎝c0a + c1a⎠    ⎠                 ⎝c0b + c1b⎠                 ⎝c

  c1a   ⎞       ⎛          ⎛   c1b   ⎞    ⎞
────────⎟ + c1b⋅⎜2.4942⋅log⎜─────────⎟ + 2⎟
0a + c1a⎠       ⎝          ⎝c0b + c1b⎠    ⎠

Gradient
⎡                   ⎛   c0a   ⎞                ⎛   c0b   ⎞               ⎛   c
⎢0, 0, 0, 2.4942⋅log⎜─────────⎟ + 2, 2.4942⋅log⎜─────────⎟, 0, 2.4942⋅log⎜────
⎣                   ⎝c0a + c1a⎠                ⎝c0b + c1b⎠               ⎝c0a 

1a   ⎞            ⎛   c1b   ⎞    ⎤
─────⎟, 2.4942⋅log⎜─────────⎟ + 2⎥
+ c1a⎠            ⎝c0b + c1b⎠    ⎦

Hessian
⎡0  0  0         0                0         0         0                0      
⎢                                               

In [4]:
f2 = min_pytential(f, vars_out=['c0', 'c1', 'Va', 'Vb'])
f2([.5,.5,.5,.5])

array([-0.7288477, -0.7288477, -0.7288477, -0.7288477])

Visualization

In [8]:
from numpy import linspace, vectorize, meshgrid
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

c0 = linspace(0.001, 0.999, 3)
va = linspace(0.001, 0.999, 3)
C0, Va = meshgrid(c0, va)
print(C0)

args_matrix = np.array([C0.flatten(), (1 - C0).flatten(), Va.flatten(), (1 - Va).flatten()])
print(args_matrix)
# Compute Z values using the fa function
F_flat = f2(args_matrix)
print(F_flat)
F = F_flat.reshape(C0.shape)

[[0.001 0.5   0.999]
 [0.001 0.5   0.999]
 [0.001 0.5   0.999]]
[[0.001 0.5   0.999 0.001 0.5   0.999 0.001 0.5   0.999]
 [0.999 0.5   0.001 0.999 0.5   0.001 0.999 0.5   0.001]
 [0.001 0.001 0.001 0.5   0.5   0.5   0.999 0.999 0.999]
 [0.999 0.999 0.999 0.5   0.5   0.5   0.001 0.001 0.001]]
[[-0.0014577 -0.7288477 -1.4562377 -0.0014577 -0.7288477 -1.4562377
  -0.0014577 -0.7288477 -1.4562377]
 [-1.4562377 -0.7288477 -0.0014577 -1.4562377 -0.7288477 -0.0014577
  -1.4562377 -0.7288477 -0.0014577]
 [-0.0014577 -0.0014577 -0.0014577 -0.7288477 -0.7288477 -0.7288477
  -1.4562377 -1.4562377 -1.4562377]
 [-1.4562377 -1.4562377 -1.4562377 -0.7288477 -0.7288477 -0.7288477
  -0.0014577 -0.0014577 -0.0014577]]


ValueError: cannot reshape array of size 36 into shape (3,3)

In [ ]:
#fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'xy'}]])
fig = go.Figure()

# Add the 3D surface plot for f
fig.add_trace(go.Surface(
    z=F, 
    x=c0, 
    y=va, 
    colorscale='Viridis',
    contours={
        "x": {"show": True, "color": "white"},
        "y": {"show": True, "color": "white"},
        "z": {"show": False, "color": "white"}
    }
))

fig.add_trace(go.Scatter3d(x=c0, y=[1]*len(c0), z=fa(c0a=c0,c1a=1-c0,Va=1), mode='lines', name='fa', line=dict(color='red', width=4)))
fig.add_trace(go.Scatter3d(x=c0, y=[0]*len(c0), z=fb(c0b=c0,c1b=1-c0,Vb=1), mode='lines', name='fb', line=dict(color='blue', width = 4)))

# # Add the 2D plot for f_reduced
# fig.add_trace(go.Scatter(x=c0, y=fa(c0a=c0,c1a=1-c0,Va=1), mode='lines', name='fa'), row=1, col=2)
# fig.add_trace(go.Scatter(x=c0, y=fb(c0b=c0,c1b=1-c0,Vb=1), mode='lines', name='fb'), row=1, col=2)
# fig.add_trace(go.Scatter(x=c0, y=f2(c0=c0,c1=1-c0,Va=1, Vb=0), mode='lines', name='fb'), row=1, col=2)

# Update layout for the entire figure
fig.update_layout(
    title='3D Surface Plot of f and 2D Plot of f_reduced',
    scene=dict(
        xaxis_title='c0',
        yaxis_title='Va',
        zaxis_title='f',
        aspectratio = dict(x=2, y=2, z=1),
        camera=dict(projection=dict(type="orthographic"))
    ),
)
fig.show()

In [ ]:
f3 = min_pytential(f, vars_out=['Va', 'Vb'])

In [ ]:

f3(Va=.5,Vb=.5)